In [33]:
import collections
import re
from pprint import PrettyPrinter
import json
import sys
import string
import unicodedata as ud
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from TwitterSearch import *
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
import twitter
import twitter.parse_tweet
import nltk
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
import random

In [10]:
def write(s):
    sys.stdout.write(s)
    sys.stdout.flush()

In [54]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [95]:
total_tweets = 300000
maxlen = 20
step = 3
epochs = 60

In [96]:
def search():
    
    tw = []
    try:
        tso = TwitterSearchOrder() # create a TwitterSearchOrder object
        tso.set_keywords([' ']) # let's define all search keywords - now, we want all tweets with a space in them
        tso.set_language('en') # we want to see english tweets only
        tso.set_include_entities(False) # and don't give us all those entity information
        ts = TwitterSearch(consumer_key = 'zg9yQTGTT2oizk3XLMHGLzfpJ',
                      consumer_secret = 'nmiwqRpWDX0oxTCUTro8sPeUVUXIZHW9O1VZcTb0mLyfHw51sc',
                      access_token = '700001043-oxm3LZ72y4WmWGRqY66QjV0SzZoHGy5OGgwic26M',
                      access_token_secret = 'hGJZWTb5bjGFSiuIQrff5UajKdlyXcp7Lyun5SJzq05Su')
        i = 0
        for tweet in ts.search_tweets_iterable(tso):
            #if (tweet['retweet_count'] != 0):
            #tw.append((len(tweet['text'].split()), tweet['retweet_count']))
            #print(str(i))
            #backspace(len(str(i)))
            tw += (tweet['text'])
            write("\rIngesting tweet %d/%d..." % (i, total_tweets))
            if i == total_tweets:
                break
            i += 1
        chars = set("".join(tw))
        return tw,chars
            #print tw
            #print( '%s: @%s tweeted: %s' % ( tweet['retweet_count'], tweet['user']['screen_name'], tweet['text'] ) )
            # print # of retweents, tweeter, and content of tweet
    except TwitterSearchException as e: # take care of all those ugly errors if there are some
        print(e)

In [97]:
text,chars = search()
text = "".join(text)

Ingesting tweet 7895/300000...

In [90]:
text

u'RT @TheLoungeKing: @BrianPardy now I go for 20% urea moisturizer which seems to work well and isn\'t gross like it sounds.RT @charsfavorite: Gifts for the holidays. /www.etsy.com/shop/Charsfavoritethings #pottiteam https://t.co/pVNZxR1LvERT @StevStiffler: This gets me every time https://t.co/CfD2iZNMNLOhhh but I liked Lou on tattoo fixers\U0001f615@RaySanityy nah niggaRT @DaydreamBR: Next year!!! https://t.co/mE38T4tfesGet Weather Updates from The Weather Channel. 18:34:54RT @kakimari: THABK YOU OMG THIS IS GORGEOUS @zapptoki https://t.co/xwmWY0x8Pb(2ne1 - I love you) el principal y el coro.\nhttps://t.co/FjfMmNOK79RT @ObeyVay_: Oh my god https://t.co/LBszorddrVRT @kezzNaughton: no matter what you do youre always going to upset somebody\U0001f644RT @QuoteBeauties: Accept.... https://t.co/SpP64My5rrRT @WotAustin: no shirt matty https://t.co/jSGz3KieUeLeadership is the art of giving people a platform for sp.eading ideas that work." - Seth Godint4 people followed me and 2 people unfollo

In [98]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
tweets = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    tweets.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(tweets))

nb sequences: 215692


In [101]:
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


In [104]:
X = np.zeros((len(tweets), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(tweets), len(chars)), dtype=np.bool)
for i, tweet in enumerate(tweets):
    for t, char in enumerate(tweet):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
len(y)

215692

In [ ]:
for iteration in range(1, epochs):
    print(y.shape)
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for iteration in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

(215692, 628)

--------------------------------------------------
Iteration 1
Epoch 1/1
  2176/215692 [..............................] - ETA: 7286s - loss: 3.7860